In [32]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import traingset as dl
import seaborn as sns

import STCGRU 
import RESGRU


In [33]:

# 定义脑区索引
from itertools import combinations

# 初始脑区定义
regions = {
    "prefrontal": [0, 1, 2, 3, 10, 11, 16],
    "central": [4, 5, 17],
    "temporal": [12, 13, 14, 15],
    "parietal": [6, 7, 18],
    "occipital": [8, 9]
}

# 前额叶_中央叶_颞叶_顶叶_枕叶

# 自动生成多脑区组合
def generate_combinations(regions, sizes):
    combined_regions = {}
    region_names = list(regions.keys())
    # 遍历指定组合大小
    for size in sizes:
        for combination in combinations(region_names, size):
            combined_name = "_".join(combination)  # 组合名称
            combined_indices = sorted(set().union(*(regions[region] for region in combination)))  # 合并去重
            combined_regions[combined_name] = combined_indices
    return combined_regions
# 生成所有二、三、四脑区组合
regions = generate_combinations(regions, sizes=[1,2, 3, 4,5])
# 动态获取变量值
name = locals()
seed = 66
dl.seed_everything(seed)

batch_size = 64
total_fold = 10  # 10折
partition = "occipital"
model_name = "stcgru_tnb"
dataset_name = "EEGData"
num_channels = len(regions[partition])
model_type = partition
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_acc_sum = 0
results_sum = []
labels_test_sum = []
results_PR_sum = []

acc_list = []
kappa_list = []
sensitivity_list = []
specificity_list = []
precision_list = []
f1_list = []

for i in range(total_fold):
    test_data_combine = torch.load(dataset_name +"/"+partition+"/TestData/test_data_"+ str(i + 1) + "_fold_with_seed_" + str(seed) + ".pth", weights_only=False)
    test_loader = DataLoader(dataset=test_data_combine,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=8)
    model = STCGRU.model(num_channels).to(device)
    # model = RESGRU.EEG_ResNet_GRU(num_channels=num_channels).to(device)
    model_loc =  model_name+'/'+partition+"/"+str(i + 1) + "_fold_model_parameter_with_seed_" + str(seed) + ".pth"
    model_dict = torch.load(model_loc,weights_only=True)
    model.load_state_dict(model_dict)
    
    '''测试'''
    acc_average, results, labels_test, results_PR = dl.model_predict(i,test_loader,model)
    results_sum.extend(results)
    labels_test_sum.extend(labels_test)
    results_PR_sum.extend(results_PR)
    name['test_acc_average_' + str(i + 1)] = acc_average

    confusion_matrix_single = confusion_matrix(labels_test, results, labels=[0, 1])
    kappa_single = cohen_kappa_score(labels_test, results)
    sensitivity_single = confusion_matrix_single[0, 0] / (confusion_matrix_single[0, 0] + confusion_matrix_single[0, 1])  # 灵敏度（召回率）
    specificity_single = confusion_matrix_single[1, 1] / (confusion_matrix_single[1, 1] + confusion_matrix_single[1, 0])  # 特异度
    precision_single = confusion_matrix_single[0, 0] / (confusion_matrix_single[0, 0] + confusion_matrix_single[1, 0])  # 查准率
    F1_single = 2 * precision_single * sensitivity_single / (precision_single + sensitivity_single)  # F1值
    acc_list.append(acc_average)
    kappa_list.append(kappa_single)
    sensitivity_list.append(sensitivity_single)
    specificity_list.append(specificity_single)
    precision_list.append(precision_single)
    f1_list.append(F1_single)

    print(model_type + " 第%d折交叉验证测试集准确率: %.4f，kappa值：%.4f，灵敏度：%.4f，特异度：%.4f，查准率：%.4f，F1值：%.4f"
        % (i+1, acc_average, kappa_single, sensitivity_single, specificity_single, precision_single, F1_single))
import pandas as pd

# 保存每一折和整体平均数据到csv
metrics_dict = {
    'Fold': [f'Fold {i+1}' for i in range(total_fold)] + ['Mean'],
    'Accuracy': acc_list + [np.mean(acc_list)],
    'Kappa': kappa_list + [np.mean(kappa_list)],
    'Sensitivity': sensitivity_list + [np.mean(sensitivity_list)],
    'Specificity': specificity_list + [np.mean(specificity_list)],
    'Precision': precision_list + [np.mean(precision_list)],
    'F1': f1_list + [np.mean(f1_list)],
}
df_metrics = pd.DataFrame(metrics_dict)
csv_path = f"result/{model_name}/{partition}_metrics.csv"
df_metrics.to_csv(csv_path, index=False)
print(f"保存至 {csv_path}")

# 保存labels_test_sum、results_sum和results_PR_sum到csv
df_pred = pd.DataFrame({
    "label": labels_test_sum,
    "prediction": results_sum,
    "prob": [x[1] if isinstance(x, (list, np.ndarray)) and len(x) > 1 else x for x in results_PR_sum]
})
pred_csv_path = f"result/{model_name}/{partition}_labels_probs.csv"
df_pred.to_csv(pred_csv_path, index=False)
print(f"保存至 {pred_csv_path}")

occipital 第1折交叉验证测试集准确率: 0.6128，kappa值：0.1984，灵敏度：0.2104，特异度：0.9776，查准率：0.8903，F1值：0.3403
occipital 第2折交叉验证测试集准确率: 0.7527，kappa值：0.5091，灵敏度：0.6814，特异度：0.8237，查准率：0.7694，F1值：0.7227
occipital 第3折交叉验证测试集准确率: 0.6936，kappa值：0.4045，灵敏度：0.7409，特异度：0.6671，查准率：0.6576，F1值：0.6968
occipital 第4折交叉验证测试集准确率: 0.7262，kappa值：0.4278，灵敏度：0.6982，特异度：0.7299，查准率：0.6908，F1值：0.6945
occipital 第5折交叉验证测试集准确率: 0.7329，kappa值：0.4576，灵敏度：0.7695，特异度：0.6921，查准率：0.6829，F1值：0.7236
occipital 第6折交叉验证测试集准确率: 0.6561，kappa值：0.2747，灵敏度：0.3374，特异度：0.9263，查准率：0.7978，F1值：0.4742
occipital 第7折交叉验证测试集准确率: 0.7316，kappa值：0.4580，灵敏度：0.8122，特异度：0.6526，查准率：0.6683，F1值：0.7333
occipital 第8折交叉验证测试集准确率: 0.6866，kappa值：0.3675，灵敏度：0.6595，特异度：0.7079，查准率：0.6606，F1值：0.6600
occipital 第9折交叉验证测试集准确率: 0.6129，kappa值：0.2177，灵敏度：0.2153，特异度：0.9908，查准率：0.9527，F1值：0.3512
occipital 第10折交叉验证测试集准确率: 0.6254，kappa值：0.2094，灵敏度：0.2168，特异度：0.9816，查准率：0.9103，F1值：0.3502
保存至 result/stcgru_tnb/occipital_metrics.csv
保存至 result/stcgru_tnb/occipital_labels_probs.csv
